In [0]:
%pip install -r requirements.txt

In [0]:
dbutils.library.restartPython()

In [0]:
import mlflow
from mlflow.models import ModelConfig

mlflow.langchain.autolog()
config = ModelConfig(development_config="config.yml")
deployment_config = ModelConfig(development_config="deployment.yml")

In [0]:
import re

def clean_string(input_string: str) -> str:
    cleaned = re.sub(r'[^a-zA-Z0-9\s]', '_', input_string)
    cleaned = re.sub(r'\s+', '_', cleaned)
    cleaned = re.sub(r'_+', '_', cleaned)
    return cleaned.strip('_').lower()


genie_space_id = config.get("genie_space_id")
_genie_agent_name = config.get("genie_agent_name")
genie_space_description = config.get("genie_space_description")

assert genie_space_id, f"Configure the genie_space_id in config.yml it is: {genie_space_id}"
assert _genie_agent_name, f"Configure the genie_agent_name in config.yml it is: {_genie_agent_name}"
assert genie_space_id, f"Configure the genie_space_description in config.yml it is: {genie_space_description}"

genie_agent_name = clean_string(_genie_agent_name)


databricks_auth_type = deployment_config.get("databricks_auth_type")
databricks_token = deployment_config.get("databricks_token")
databricks_host = deployment_config.get("databricks_host")
assert databricks_auth_type, f"Configure the databricks_auth_type in deployment.yml it is: {databricks_auth_type}"
assert databricks_token, f"Configure the databricks_token in deployment.yml it is: {databricks_token}"
assert databricks_host, f"Configure the databricks_host in deployment.yml it is: {databricks_host}"

In [0]:
# Log the model to MLflow
import os
import mlflow
from mlflow.models.resources import DatabricksServingEndpoint, DatabricksGenieSpace, DatabricksSQLWarehouse

input_example = {
    "messages": [
        {
            "role": "user",
            "content": "What is an LLM agent and what is 2 + 2?"
        }
    ]
}

with mlflow.start_run():
    logged_agent_info = mlflow.langchain.log_model(
        lc_model=os.path.join(
            os.getcwd(),
            'model',
        ),
        pip_requirements="requirements.txt",
        model_config="config.yml",
        artifact_path='model',
        input_example=input_example,
    )

In [0]:
import pandas as pd

eval_examples = [
    {
        "request": {
            "messages": [
                {
                    "role": "user",
                    "content": "What is 2+2?"
                }
            ]
        },
        "expected_response": None
    }
]

eval_dataset = pd.DataFrame(eval_examples)
display(eval_dataset)

In [0]:
import mlflow

with mlflow.start_run(run_id=logged_agent_info.run_id):
    eval_results = mlflow.evaluate(
        f"runs:/{logged_agent_info.run_id}/model",  # replace `chain` with artifact_path that you used when calling log_model.
        data=eval_dataset,  # Your evaluation dataset
        model_type="databricks-agent",  # Enable Mosaic AI Agent Evaluation
    )

# Review the evaluation results in the MLFLow UI (see console output), or access them in place:
display(eval_results.tables['eval_results'])

In [0]:
from databricks.sdk import WorkspaceClient
import re

w = WorkspaceClient()

def normalize(inp: str) -> str:
    normalized_prefix = re.sub(r'\W+', '_', inp)
      # Remove multiple underscores in a row
    return re.sub(r'_+', '_', normalized_prefix).strip("_")

def get_current_user_normalized():
  email = w.current_user.me().user_name.split("@")[0]
  return normalize(email)

prefix = get_current_user_normalized()
prefix

In [0]:
import os

current_directory = os.getcwd()
current_folder_name = os.path.basename(current_directory)
normalized_current_folder = normalize(current_folder_name)
normalized_current_folder

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model
catalog = "main"
schema = "default"
# TODO: modify these values
model_name = f"{prefix}_{normalized_current_folder}"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME)

In [0]:
from databricks import agents

# Deploy the model to the review app and a model serving endpoint
deployment = agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version, environment_vars={
  "DATABRICKS_AUTH_TYPE": databricks_auth_type,
  "DATABRICKS_TOKEN": databricks_token,
  "DATABRICKS_HOST": databricks_host
})

In [0]:
deployment

In [0]:
result = w.serving_endpoints.wait_get_serving_endpoint_not_updating(name=deployment.endpoint_name, )

In [0]:
if result.state.config_update.value == "UPDATE_FAILED":
    raise Exception("Deployment failed")

In [0]:
from databricks.sdk.service.serving import ChatMessage, ChatMessageRole
messages = [ChatMessage(role=ChatMessageRole.USER, content="what are the top 10 companies that are most and least consuming when it comes to CO2? what about electricity or fuel?"),]
w.serving_endpoints.query(name=deployment.endpoint_name, messages=messages)

In [0]:
is_int_testing_job = None
try:
    is_int_testing_job = dbutils.widgets.get(name="int_testing_job")
except Exception:
    pass

if is_int_testing_job is not None:
    state = w.serving_endpoints.delete(deployment.endpoint_name)
    print(state)